# Trademark design code comparison

_Compares computer generated design codes (predicted) to Examiner selected design codes (actual)

In [ ]:
import csv
p_file = 'trademarks_dsc_predicted.csv'
a_file = 'trademarks_dsc_actuals.csv'

In [ ]:
import csv
p_file = '2018_trademark_dsc_predicted.csv'
a_file = 'actuals_10596trademark_dsc.csv'

In [ ]:
import csv
p_file = '2018_trademark_dsc_predicted.csv'
a_file = '2018_trademark_dsc_actuals.csv'

In [ ]:
# read in csv data and write to a dictionary
p_data = {}
with open(p_file) as fin:
    reader = csv.reader(fin, skipinitialspace=True)
#     remove if no header
    next(reader)
    for row in reader:
        p_data[row[0]]=set(row[1:])

In [ ]:
# # read in csv data and write to a dictionary
a_data = {}
with open(a_file) as fin:
    reader = csv.reader(fin, skipinitialspace=True)
#     remove if no header
    next(reader)
    for row in reader:
        a_data[row[0]]=set(row[1:])
# print('Original dict:')
# print(a_data)        
# filter a_data to remove 26 series codes

exclude = ('26.', '02.', '.10')
# f_a_data = {}
# for k, v in a_data.items():
#     f_a_data.setdefault(k,[])
#     for code in v:
#         print (k, code, not code.startswith(exclude))
#         if not code.startswith(exclude):
#             f_a_data[k].append(code)
        
            
# print('Filtered dict:')
# print(f_a_data)    

f_a_data = {k:{x for x in v if all(s not in x for s in exclude)} for k,v in a_data.items()}
# print(f_a_data)

# if you want to process the filtered actual data, uncomment following line
a_data = f_a_data

print(a_data)

## Run this cell to print the results to the screen

In [ ]:
# run this cell to print results to screen
missed_list =[]
s_rate ={}
# v & p_data[k] is the intersection
# v - p_data[k] is in a_data only
# p_data[k]-v is in p_data only
results = {k: (v & p_data[k], v - p_data[k], p_data[k]-v) for k, v in a_data.items() if k in p_data}

for key, value in results.items():
    print(f'For {key}: in predicted & actual: {value[0]}, in only actual: {value[1]}, in only predicted: {value[2]}')
    missed_list.append(value[1])
for key, value in results.items():
    if (len(value[0]) + len(value[1])) == 0:
        s_rate[keyT] = 1
    else:
        s_rate[key] = (len(value[0]) / (len(value[0]) + len(value[1])))
print(type(s_rate.keys()))

for key, value in s_rate.items():
    print(f'In application {key}, found {value*100}%')
flat_missed_list = [item for sublist in missed_list for item in sublist]
# print(flat_missed_list)

### Counts & prints the missed design codes 

In [ ]:
from collections import Counter
c = Counter(flat_missed_list)
# print(c.most_common(10))
c_short = c.most_common(20)

# convert diction to two list for plotting
keys = []
values = []
for k, v in c_short:
    keys.append(k)
    values.append(v)

# print(keys, values)
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20,10))
sns.barplot(y=values, x=keys, alpha=.9, palette='PuBuGn_d')
plt.title('Most often missed design codes')
plt.ylabel('#', fontsize=12)
plt.xlabel('design code', fontsize=12)
plt.show()

### Plots bar graph of success rate for applications (do not run if looking at more than 10 applications)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# convert diction to two list for plotting
keys = []
values = []
for k, v in s_rate.items():
    keys.append(k)
    values.append(v)

# print(keys, values)

plt.figure(figsize=(20,15))
sns.barplot(y=values, x=keys, alpha=.9, palette='PuBuGn_d')
plt.title('Success Rate')
plt.ylabel('%', fontsize=12)
plt.xlabel('Application Number', fontsize=12)
plt.show()

## Run this cell to write results to files

In [ ]:
# run this cell to write results to files
missed_list =[]
s_rate ={}
# v & p_data[k] is the intersection
# v - p_data[k] is in a_data only
# p_data[k]-v is in p_data only
results = {k: (v & p_data[k], v - p_data[k], p_data[k]-v) for k, v in a_data.items() if k in p_data}

with open('results_verbose.csv', 'w') as fout:
    for key, value in results.items():
        fout.write(f'For {key}: in predicted & actual: {value[0]}, Not predicted: {value[1]}, extra predictions: {value[2]}\n')
        missed_list.append(value[1])
for key, value in results.items():
    if (len(value[0]) + len(value[1])) == 0:
        s_rate[key] = 1
    else:
        s_rate[key] = (len(value[0]) / (len(value[0]) + len(value[1])))

with open('results_simple.csv', 'w') as fout:
    for key, value in s_rate.items():
        fout.write(f'{key},{value}\n')
flat_missed_list = [item for sublist in missed_list for item in sublist]
# print(flat_missed_list)

In [ ]:
from collections import Counter
c = Counter(flat_missed_list)
# print(c.most_common(10))
c_short = c.most_common()
print(c_short)
# convert diction to two list for plotting
keys = []
values = []
for k, v in c_short:
    keys.append(k)
    values.append(v)

# print(keys, values)
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,10))
sns.barplot(y=values, x=keys, alpha=.9, palette='PuBuGn_d')
plt.title('Most often missed design codes')
plt.ylabel('#', fontsize=12)
plt.xlabel('design code', fontsize=12)
plt.show();

# show most common missed except for:
excluded_flat_missed_list = []
exclude = ('26', '27')
for code in flat_missed_list:
    if not code.startswith(exclude):
        excluded_flat_missed_list.append(code)
xc = Counter(excluded_flat_missed_list)
print(xc.most_common(20))

In [ ]:
import pandas as pd

with open('results_simple.csv', 'r') as fin:
    df = pd.read_csv(fin, header=None, names=['serial','percent'])

print(df['percent'].describe())
df['percent'].hist(bins=5)
print(df['percent'].value_counts(normalize=True).head(10))

In [ ]:
import pandas as pd

xdf = pd.DataFrame(excluded_flat_missed_list, columns=['codes'])
print(xdf['codes'].describe())
print(xdf['codes'].value_counts(normalize=True).head(10))
